# Ryhmä-190

## Python-paketit

In [100]:
import pandas as pd
import numpy as np
from scipy.stats import boxcox, yeojohnson
from sklearn.metrics import mean_squared_error, make_scorer, r2_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.discriminant_analysis import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR
import optuna

# Hyödylliset funktiot

In [101]:
def add_meaningful_features(orig_df):
    df = orig_df.copy()
    df = df.replace([np.inf, -np.inf], np.nan).fillna(0)

    df['AtomFraction'] = (df['NumOfC'] + df['NumOfO'] + df['NumOfN']) / df['NumOfAtoms']
    df['Polarity'] = df['NumHBondDonors'] / df['MW']
    df['HBondDensity'] = df['NumHBondDonors'] / df['NumOfAtoms']
    df['GroupDensity_CarboxylicAcid'] = df['carboxylic acid'] / df['MW']
    df['Unsaturation'] = df['C=C (non-aromatic)'] + df['C=C-C=O in non-aromatic ring']
    df['ConfigurationalComplexity'] = df['NumOfConf'] / df['MW']

    df = pd.get_dummies(df, columns=['parentspecies'], drop_first=True)
    df['HydrogenBondPotential'] = df['NumHBondDonors'] + (df['NumOfO'] + df['NumOfN'])
    polar_groups = ['hydroxyl (alkyl)', 'aldehyde', 'ketone', 'carboxylic acid', 'ester', 'nitro']
    df['PolarGroupCount'] = df[polar_groups].sum(axis=1)
    df['AromaticGroupFraction'] = df['aromatic hydroxyl'] / (df[polar_groups + ['aromatic hydroxyl']].sum(axis=1) + 1e-9)
    df['MolecularSize'] = df['MW'] + df['NumOfAtoms']
    df['DegreeOfUnsaturation'] = df['NumOfC'] + 1 - (df['NumOfAtoms'] - df['NumOfC']) / 2
    df['FlexibilityRatio'] = df['NumOfConfUsed'] / (df['NumOfConf'] + 1e-9)
    df['PolarityIndex'] = (df['NumOfO'] + df['NumOfN'] + df[polar_groups].sum(axis=1)) / (df['NumOfC'] + df['C=C (non-aromatic)'] + 1e-9)
    df['Hydrophobicity'] = df['NumOfC'] / (df['NumOfAtoms'] + 1e-9)
    functional_groups = [
        'C=C (non-aromatic)', 'hydroxyl (alkyl)', 'aldehyde', 'ketone',
        'carboxylic acid', 'ester', 'ether (alicyclic)', 'nitrate',
        'nitro', 'aromatic hydroxyl', 'carbonylperoxynitrate', 'peroxide',
        'hydroperoxide', 'carbonylperoxyacid', 'nitroester'
    ]
    for group in functional_groups:
        df[f'{group}_Indicator'] = (df[group] > 0).astype(int)

    symmetric_groups = ['ester', 'ether (alicyclic)', 'ketone']
    asymmetric_groups = ['aldehyde', 'carboxylic acid']
    df['SymmetryIndex'] = df[symmetric_groups].sum(axis=1) / (df[symmetric_groups + asymmetric_groups].sum(axis=1) + 1e-9)
    df['ShapeCompactness'] = df['NumOfAtoms'] / (df['NumOfConfUsed'] + 1e-9)
    df['HydrogenBondDensity'] = df['NumHBondDonors'] / (df['MW'] + 1e-9)
    df['VolatilityIndex'] = (
        df['NumOfC'] / (df[polar_groups].sum(axis=1) + 1e-9) *
        1 / (df['MW'] + 1e-9)
    )
    df['OxygenToCarbonRatio'] = df['NumOfO'] / (df['NumOfC'] + 1e-9)

    return df

In [102]:
trans_functions = {
    'none': lambda x: x,
    'log': lambda x: np.log(x + 1e-9),
    'sqrt': lambda x: np.sqrt(x),
    'square': lambda x: x**2,
    'cube': lambda x: x**3,
    'exp': lambda x: np.exp(x),
    'reciprocal': lambda x: 1 / (x + 1e-9),
    'boxcox': lambda x: boxcox(x + 1e-9)[0] if (x > 0).all() else x,
    'yeojohnson': lambda x: yeojohnson(x)[0]
}

def apply_transformations(df, selected_transformations):    
    df_transformed = df.copy()
    for col, func in selected_transformations:
        if func in trans_functions:
            df_transformed[col] = trans_functions[func](df_transformed[col].astype(float))
    
    return df_transformed

In [103]:
def evaluate_model(model, X_train, y_train, random_state=42):
    ### Train loss
    y_train_pred = model.predict(X_train)
    train_loss = mean_squared_error(y_train, y_train_pred)

    ### 5-fold cross-validation
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    mse_scorer = make_scorer(mean_squared_error)
    cv_scores = cross_val_score(model, X_train, y_train, cv=kf, scoring=mse_scorer)
    cv_loss_mean = cv_scores.mean()

    r2_train = r2_score(y_train, y_train_pred)
    r2_cv = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2').mean()
    
    return train_loss, cv_loss_mean, r2_train, r2_cv

In [104]:
best_transformations = [
    ('AtomFraction', 'cube'),
    #('GroupDensity_CarboxylicAcid', 'sqrt'),
    ('ConfigurationalComplexity', 'yeojohnson'),
    ('ShapeCompactness', 'log'),
    ('VolatilityIndex', 'reciprocal'),
    ('FlexibilityRatio', 'sqrt'),
    ('PolarityIndex', 'sqrt'),
    ('NumOfConfUsed', 'boxcox'),
    ('DegreeOfUnsaturation', 'yeojohnson'),
    ('Hydrophobicity', 'yeojohnson'),
    ('HydrogenBondPotential', 'none'),
    ('HBondDensity', 'none'),
    ('SymmetryIndex', 'sqrt'),
    ('NumOfConf', 'sqrt'),
    ('NumOfC', 'cube'),
    ('NumOfO', 'sqrt'),
]
#New best combination: ['AtomFraction', 'GroupDensity_CarboxylicAcid', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7551643425
#New best combination: ['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7553619528
#New best combination: ['AtomFraction', 'GroupDensity_CarboxylicAcid', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7553724869
#New best combination: ['AtomFraction', 'GroupDensity_CarboxylicAcid', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7553937006
#New best combination: ['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7555560201
#New best combination: ['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator'] with score: 0.7555890842
# best_features = ['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator']
best_features = ['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator']

## Tietoaineistojen lataaminen

In [105]:
df_train = pd.read_csv('../data/train.csv', encoding='utf-8', header=0)
df_test = pd.read_csv('../data/test.csv', encoding='utf-8', header=0)

## Datan myllääminen

In [106]:
# Erotetaan X ja y
X_train, y_train = df_train.drop(columns=['log_pSat_Pa', 'ID']), df_train['log_pSat_Pa']
X_test = df_test.drop(columns=['ID'])

# Lisätään uusia featureita
X_train_trans = add_meaningful_features(X_train)
X_test_trans = add_meaningful_features(X_test)

# Käytetään vain parhaita featureita
X_train_trans = X_train_trans[best_features]
X_test_trans = X_test_trans[best_features]

# Käytetään vain parhaita transformaatioita
X_train_trans = apply_transformations(X_train_trans, selected_transformations=best_transformations)
X_test_trans = apply_transformations(X_test_trans, selected_transformations=best_transformations)


# Hyperparametrien optimointi

In [114]:
def objective(trial):
    svr_params = {        
        "C": trial.suggest_float("C", 4.6, 5.5),
        "epsilon": trial.suggest_float("epsilon", 0.35, 0.45),
        "kernel": 'rbf',
        "degree": trial.suggest_int("degree", 140, 190),
        "gamma": trial.suggest_float("gamma", 0.010, 0.017),
        "coef0": trial.suggest_float("coef0", 0.015, 0.029),
        #"shrinking": trial.suggest_categorical("shrinking", [True, False]),
        "tol": trial.suggest_float("tol", 1.4e-5, 2.8e-5),
        #"cache_size": 200,
        #"verbose": False,
        #"max_iter": -1,
    }

    model = make_pipeline(StandardScaler(), SVR(**svr_params))
    #score = evaluate_model(model, X_train_trans, y_train)[3]
    score = cross_val_score(model, X_train_trans, y_train, cv=5, scoring='r2').mean()
    return score

study_name = "group-190-667f"
storage = "sqlite:///optuna_190.sqlite3"

study = optuna.create_study(
    direction="maximize",
    #sampler=optuna.samplers.TPESampler(seed=190),
    study_name=study_name,
    storage=storage,
    load_if_exists=True
)

study.optimize(objective, n_trials=100)

loaded_study = optuna.load_study(study_name=study_name, storage=storage)

print(f"The best score: {loaded_study.best_value}")
print(f"The best hyperparameter combination: {loaded_study.best_params}")

# Trial 525 finished with value: 0.7556880608718612 and parameters: {'C': 4.317548086868634, 'epsilon': 0.4029669979799471, 'degree': 128, 'gamma': 0.014430936891064592, 'coef0': 0.0263038871779371, 'tol': 1.1502467635687632e-05}. Best is trial 525 with value: 0.7556880608718612.
# Trial 557 finished with value: 0.7557184013541394 and parameters: {'C': 4.428631164497283, 'epsilon': 0.40534818155621216, 'degree': 129, 'gamma': 0.014189442659395292, 'coef0': 0.027242821178532224, 'tol': 1.613436864917458e-05}. Best is trial 557 with value: 0.7557184013541394.
# Trial 667 finished with value: 0.7558252623881684 and parameters: {'C': 4.796893660934806, 'epsilon': 0.41591433032412095, 'degree': 150, 'gamma': 0.013649066977714302, 'coef0': 0.025826880204001527, 'tol': 1.6879135343901046e-05}. Best is trial 667 with value: 0.7558252623881684.
# Trial 713 finished with value: 0.7558501480076878 and parameters: {'C': 4.9675057323406, 'epsilon': 0.41996635414854416, 'degree': 169, 'gamma': 0.013344940612796844, 'coef0': 0.022940696618095578, 'tol': 2.3855715372445768e-05}. Best is trial 713 with value: 0.7558501480076878.

[I 2024-12-02 15:35:46,278] Using an existing study with name 'group-190-667f' instead of creating a new one.
[I 2024-12-02 15:44:23,359] Trial 704 finished with value: 0.7557973577110069 and parameters: {'C': 4.9161924167783875, 'epsilon': 0.4182563381239965, 'degree': 170, 'gamma': 0.013209887002347516, 'coef0': 0.02252786631514679, 'tol': 2.3544265858185934e-05}. Best is trial 696 with value: 0.7558452610703278.
[I 2024-12-02 15:51:48,155] Trial 705 finished with value: 0.7558231373464939 and parameters: {'C': 4.864163477565651, 'epsilon': 0.42010043714850687, 'degree': 167, 'gamma': 0.013426664119896327, 'coef0': 0.022361397509036075, 'tol': 2.232073352824151e-05}. Best is trial 696 with value: 0.7558452610703278.
[I 2024-12-02 15:55:31,323] Trial 706 finished with value: 0.7558403545608121 and parameters: {'C': 4.903601246558182, 'epsilon': 0.41971422769904115, 'degree': 168, 'gamma': 0.013442254496700841, 'coef0': 0.022350350536630023, 'tol': 2.2400865662571464e-05}. Best is tria

KeyboardInterrupt: 

In [108]:
loaded_study = optuna.load_study(study_name=study_name, storage=storage)
best_params = loaded_study.best_params
print(f"Training with best params: {best_params}")
model = make_pipeline(StandardScaler(), SVR(**best_params))
model.fit(X_train_trans, y_train)
score = evaluate_model(model, X_train_trans, y_train)
print(score)

Training with best params: {'C': 4.796893660934806, 'epsilon': 0.41591433032412095, 'degree': 150, 'gamma': 0.013649066977714302, 'coef0': 0.025826880204001527, 'tol': 1.6879135343901046e-05}
(np.float64(2.1953460261306685), np.float64(2.380399534028803), 0.7744946628945297, np.float64(0.7553174277875062))


## Ennustuksen tallentaminen

In [109]:
df_test['log_pSat_Pa'] = model.predict(X_test_trans)
df_test[['ID', 'log_pSat_Pa']].to_csv('../submission/submission.csv', index=False)

# Lisää työkaluja

In [ ]:
import itertools
import sys

def evaluate_model(model, X, y):
    scores = cross_val_score(model, X, y, cv=5, scoring='r2')
    return scores.mean()

#always_include = ['AtomFraction', 'GroupDensity_CarboxylicAcid', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'nitro', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator']
always_include = []
remaining_cols = [col for col in X_train_trans.columns if col not in always_include]
print(remaining_cols)

loaded_study = optuna.load_study(study_name=study_name, storage=storage)
best_params = loaded_study.best_params
model = make_pipeline(StandardScaler(), SVR(**best_params))

best_score = -np.inf
best_combination = None

mode = 'interaction'

if mode == 'add':
    for L in range(0, len(remaining_cols) + 1):
        for subset in itertools.combinations(remaining_cols, L):
            current_combination = list(always_include) + list(subset)
            X_subset = X_train_trans[current_combination]
            model.fit(X_subset, y_train)
            score = evaluate_model(model, X_subset, y_train)
            
            if score > best_score:
                best_score = score
                best_combination = current_combination
                print(f'New best combination: {best_combination} with score: {best_score:.10f}')
elif mode == 'remove':
    X_transformed = X_train_trans.copy()
    for L in range(0, len(always_include) + 1):
        for subset in itertools.combinations(always_include, L):
            current_combination = [col for col in always_include if col not in subset]
            X_subset = X_transformed[current_combination]
            model.fit(X_subset, y_train)
            score = evaluate_model(model, X_subset, y_train)
            
            if score > best_score:
                best_score = score
                best_combination = current_combination
                print(f'New best combination: {best_combination} with score: {best_score:.10f}')
elif mode == "replace":
    for included_col in always_include:
        for excluded_col in remaining_cols:
            current_combination = [col for col in always_include if col != included_col] + [excluded_col]
            X_subset = X_train_trans[current_combination]
            model.fit(X_subset, y_train)
            score = evaluate_model(model, X_subset, y_train)
            
            if score > best_score:
                best_score = score
                best_combination = current_combination
                print(f'New best combination: {best_combination} with score: {best_score:.10f}')
elif mode == "transform":
    X_subset = X_train_trans[always_include]
    model.fit(X_subset, y_train)
    baseline_score = evaluate_model(model, X_subset, y_train)
    print(f'Baseline score with no transformation: {baseline_score:.10f}')
    
    ### Käydään kaikki transformaatiot läpi
    for col in always_include:
        if any(col == approved_col for approved_col, _ in best_transformations):
            continue  # Skip columns with approved transformations
        if X_train_trans[col].nunique() < 10:
            continue
        for name, transform in trans_functions.items():
            try:
                X_temp = X_train_trans.copy()
                X_temp[col] = transform(X_temp[col].astype(float))
                
                if np.any(np.isinf(X_temp[col])) or np.any(np.abs(X_temp[col]) > 1e10):
                    raise ValueError(f"Transformation {name} for {col} resulted in infinities or excessively large values.")
                
                X_subset = X_temp[always_include]
                model.fit(X_subset, y_train)
                score = evaluate_model(model, X_subset, y_train)
                print(f'Transformation: {col} with {name} transformation, score: {score:.10f}')
                if score > best_score:
                    best_score = score
                    best_combination = (col, name)
                    #print(f'New best transformation: {col} with {name} transformation, score: {best_score:.10f}')
            except Exception as e:
                print(f"Skipping transformation {name} for {col} due to error: {e}")
elif mode == "interaction":
    for col1, col2 in itertools.combinations(remaining_cols, 2):
        if X_train_trans[col1].nunique() < 10 or X_train_trans[col2].nunique() < 10:
            continue
        
        X_temp = X_train_trans.copy()
        interaction_term = X_temp[col1] * X_temp[col2]
        X_temp['interaction'] = interaction_term
        
        # Test with interaction term without dropping original variables
        X_subset = X_temp[remaining_cols + ['interaction']]
        model.fit(X_subset, y_train)
        score = evaluate_model(model, X_subset, y_train)
        print(f'Interaction: {col1} * {col2} without dropping, score: {score:.10f}')
        if score > best_score:
            best_score = score
            best_combination = (col1, col2, 'without dropping')
        
        # Test with interaction term and dropping original variables
        X_subset = X_temp[remaining_cols + ['interaction']]
        X_subset = X_subset.drop(columns=[col1, col2])
        model.fit(X_subset, y_train)
        score = evaluate_model(model, X_subset, y_train)
        print(f'Interaction: {col1} * {col2} with dropping, score: {score:.10f}')
        if score > best_score:
            best_score = score
            best_combination = (col1, col2, 'with dropping')

print(f'Best combination: {best_combination} with score: {best_score:.10f}')

# baseline: 0.7558501480076878

['AtomFraction', 'ConfigurationalComplexity', 'carbonylperoxynitrate_Indicator', 'PolarityIndex', 'parentspecies_apin', 'aldehyde_Indicator', 'peroxide_Indicator', 'ShapeCompactness', 'FlexibilityRatio', 'VolatilityIndex', 'PolarGroupCount', 'HBondDensity', 'HydrogenBondPotential', 'nitro_Indicator', 'ester_Indicator', 'parentspecies_toluene', 'DegreeOfUnsaturation', 'Hydrophobicity', 'ketone_Indicator', 'nitroester_Indicator', 'hydroxyl (alkyl)', 'carbonylperoxyacid', 'NumOfConfUsed', 'aldehyde', 'SymmetryIndex', 'NumOfConf', 'NumOfC', 'C=C (non-aromatic)', 'C=C (non-aromatic)_Indicator', 'nitrate', 'carbonylperoxyacid_Indicator', 'hydroperoxide_Indicator', 'nitroester', 'carboxylic acid_Indicator', 'aromatic hydroxyl_Indicator', 'NumOfO', 'ether (alicyclic)_Indicator']
Interaction: AtomFraction * ConfigurationalComplexity without dropping, score: 0.7560278346
Interaction: AtomFraction * ConfigurationalComplexity with dropping, score: 0.7553295516
Interaction: AtomFraction * PolarityI